In [ ]:
import pandas as pd
from sqlalchemy import create_engine, text
import cred_pg as c

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

In [ ]:
engine = create_engine(
    f'postgresql://{c.pg_userid}:{c.pg_password}@{c.pg_host}/{c.pg_db}', 
    connect_args = {'options': '-c search_path=umobility,public', 'keepalives_idle': 120},
    pool_size=1, 
    max_overflow=0,
    execution_options={ 'isolation_level': 'AUTOCOMMIT' }
)

In [ ]:
def exe(sql):
    with engine.connect() as con:
        return pd.read_sql_query(text(sql), con)

In [ ]:
sql = """
select
  relname as table_name,
  n_live_tup as row_count
from pg_stat_user_tables
where schemaname = 'umobility'
order by schemaname, relname;
"""
exe(sql)

In [ ]:
sql = """
with recursive 
paths(startnode, endnode, path) as (
   select
        hid_a as startnode,
        hid_b as endnode,
        array[hid_a, hid_b] as path
     from segment_bi
     where hid_a = 10152 -- Heidelberger Platz
   union all
   select 
        paths.startnode as startnode,
        hid_b as endnode,
        array_append(path, hid_b) as path
     from paths
     join segment_bi on paths.endnode = hid_a
   where not hid_b=any(paths.path)
),
paths1 as (
  select startnode, endnode, unnest(path) as hid
  from paths
),
paths2 as (
  select startnode, endnode, paths1.hid, h.bez
  from paths1
    join haltestelle h using(hid)
)
select startnode, endnode,  array_agg(bez) as hops
from paths2
group by startnode, endnode
order by count(*), hops;
"""
df = exe(sql)
df

In [ ]:
for r in df.hops.items():
    print(r)

In [30]:
df.iat[9,2]

['HeidelbergerPlatz',
 'FehrbellinerPlatz',
 'Hohenzollernplatz',
 'SpichernStr',
 'Kurfürstendamm',
 'ZoologischerGarten',
 'HeidelbergerPlatz',
 'FehrbellinerPlatz',
 'Hohenzollernplatz',
 'SpichernStr',
 'Kurfürstendamm',
 'Wittenbergplatz',
 'ZoologischerGarten',
 'HeidelbergerPlatz',
 'FehrbellinerPlatz',
 'Hohenzollernplatz',
 'SpichernStr',
 'AugsburgerStr',
 'Wittenbergplatz',
 'ZoologischerGarten',
 'HeidelbergerPlatz',
 'FehrbellinerPlatz',
 'BlisseStr',
 'BerlinerStr',
 'GüntzelStr',
 'SpichernStr',
 'Kurfürstendamm',
 'ZoologischerGarten',
 'HeidelbergerPlatz',
 'FehrbellinerPlatz',
 'Hohenzollernplatz',
 'SpichernStr',
 'AugsburgerStr',
 'Wittenbergplatz',
 'Kurfürstendamm',
 'ZoologischerGarten',
 'HeidelbergerPlatz',
 'FehrbellinerPlatz',
 'KonstanzerStr',
 'Adenauerplatz',
 'WilmersdorferStr',
 'BismarckStr',
 'DeutscheOper',
 'ErnstReuterPlatz',
 'ZoologischerGarten',
 'HeidelbergerPlatz',
 'FehrbellinerPlatz',
 'BlisseStr',
 'BerlinerStr',
 'GüntzelStr',
 'SpichernStr